# Processing mobility data from Descartes Labs

In [1]:
# https://github.com/dnanhkhoa/nb_black
%load_ext lab_black

In [2]:
import altair as alt
import pandas as pd
import sys
import json
import glob
import base64
import numpy as np
from collections import OrderedDict
from operator import getitem
import datetime as dt
from datetime import timedelta
from io import BytesIO
import matplotlib.pyplot as plt
from matplotlib import dates
from matplotlib.dates import WeekdayLocator, DayLocator, DateFormatter, date2num
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
import logging
from datetime import datetime
import altair_latimes as lat

In [3]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")

ThemeRegistry.enable('latimes')

### Fetch the ndjson 

In [4]:
df = pd.read_json(
    "../../DL-COVID-19/DL-us-mobility.ndjson", lines=True, dtype={"fips": str}
)

### Fetch csv format from local copy of Decartes Labs' [repo](https://github.com/descarteslabs/DL-COVID-19)

In [5]:
# Technical report: https://arxiv.org/pdf/2003.14228.pdf

In [6]:
src = pd.read_csv(
    "../../DL-COVID-19/DL-us-mobility-daterow.csv",
    dtype={"fips": str, "admin_level": str},
    parse_dates=True,
    infer_datetime_format=True,
).fillna("")

In [7]:
src["date"] = pd.to_datetime(src["date"])

In [8]:
src["weekday"] = src["date"].dt.day_name()

---

### Filter the dataframe to California and slim

In [9]:
ca_state = (
    src[(src["admin1"] == "California") & (src["admin_level"] == "1")]
    .drop(["country_code", "admin_level", "samples"], axis=1)
    .rename(columns={"admin2": "county"})
)

In [10]:
ca_counties = (
    src[(src["admin1"] == "California") & (src["admin_level"] == "2")]
    .drop(["country_code", "admin_level", "admin1", "samples"], axis=1)
    .rename(columns={"admin2": "county"})
)

In [11]:
ca_counties["county"] = ca_counties["county"].str.replace(" County", "")

In [12]:
ca_counties.head()

,date,county,fips,m50,m50_index,weekday
49651,2020-03-01,Alameda,06001,3.684,58,Sunday
49652,2020-03-02,Alameda,06001,6.182,98,Monday
49653,2020-03-03,Alameda,06001,6.271,100,Tuesday
49654,2020-03-04,Alameda,06001,6.209,99,Wednesday
49655,2020-03-05,Alameda,06001,6.654,106,Thursday


---

### Filter the dataframe to counties in our region

In [13]:
socal = [
    "Los Angeles",
    "Ventura",
    "San Bernardino",
    "Riverside",
    "Orange",
]

In [14]:
socal_counties = ca_counties[ca_counties["county"].isin(socal)]

In [15]:
socal_counties.head()

,date,county,fips,m50,m50_index,weekday
54751,2020-03-01,Los Angeles,06037,3.996,60,Sunday
54752,2020-03-02,Los Angeles,06037,6.502,98,Monday
54753,2020-03-03,Los Angeles,06037,6.591,99,Tuesday
54754,2020-03-04,Los Angeles,06037,6.595,100,Wednesday
54755,2020-03-05,Los Angeles,06037,7.008,106,Thursday


---

### Filter the dataframe to Los Angeles County

In [23]:
la_county = socal_counties[socal_counties["county"] == "Los Angeles"]

In [24]:
sb_county = socal_counties[socal_counties["county"] == "San Bernardino"]

In [25]:
la_county.head()

,date,county,fips,m50,m50_index,weekday
54751,2020-03-01,Los Angeles,06037,3.996,60,Sunday
54752,2020-03-02,Los Angeles,06037,6.502,98,Monday
54753,2020-03-03,Los Angeles,06037,6.591,99,Tuesday
54754,2020-03-04,Los Angeles,06037,6.595,100,Wednesday
54755,2020-03-05,Los Angeles,06037,7.008,106,Thursday


In [48]:
alt.Chart(la_county).mark_bar().encode(
    x=alt.X("date:T", axis=alt.Axis(format="%B", title=" ", tickCount=3)),
    y=alt.Y(
        "m50",
        title=" ",
        axis=alt.Axis(format="", title="", tickCount=6),
    ),
    facet=alt.Facet("county"),
).properties(width=1000, height=300, title="Mobility in Los Angeles County")

alt.Chart(...)

---

### Graph the m50 figure, by county

In [47]:
alt.Chart(socal_counties).mark_area(color="red").encode(
    x=alt.X("date:T", axis=alt.Axis(format="%B", title=" ", tickCount=3)),
    y=alt.Y(
        "m50",
        title=" ",
        axis=alt.Axis(format="2s", title="", tickCount=6),
    ),
    facet=alt.Facet("county", columns=5),
).properties(width=200, height=200, title="Average mobility (km) in SoCal Counties")

alt.Chart(...)